- Environment name: opioid-eda

# Data

## Download

In [1]:
# !pip install kaggle
# !kaggle datasets download -d craigchilvers/opioids-in-canada
# !unzip opioids-in-canada.zip

opioids-in-canada.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  opioids-in-canada.zip
replace Canada opioid and stimulant Subtance Harms Data June 2022.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [ ]:
# load the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

,Province name,Population (2021 Census),Land area (km^2),Population per km^2,"GDP (million CAD, 2019)","GDP per capita (CAD, 2019)","Market income per capita (CAD, 2019)"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ontario,"14,223,942","908,699.33",15.653079,"891,811","54,149","36,298"
2,Quebec,"8,501,833","1,356,625.27",6.266899,"460,357","54,149","32,351"
3,British Columbia,"5,000,879","922,503.01",5.420989,"309,059","60,707","36,723"
4,Alberta,"4,262,635","640,330.46",6.656930,"352,884","80,905","40,094"


# Examine Canadian Population and GDP Statistics Data

In [ ]:
df_can_stats = pd.read_csv('Canada population and GDP by province.csv')
df_can_stats.head()

In [ ]:
# check for missing values
df_can_stats.isnull().sum()

Province name                           1
Population (2021 Census)                1
Land area (km^2)                        1
Population per km^2                     1
GDP (million CAD, 2019)                 1
GDP per capita (CAD, 2019)              1
Market income per capita (CAD, 2019)    1
dtype: int64

In [ ]:
# check rows with missing values
missing_values = df_can_stats.isnull().any(axis=1)
rows_with_missing_values = df_can_stats[missing_values]
print(rows_with_missing_values)

  Province name Population (2021 Census) Land area (km^2)   
0           NaN                      NaN              NaN  \

   Population per km^2 GDP (million CAD, 2019) GDP per capita (CAD, 2019)   
0                  NaN                     NaN                        NaN  \

  Market income per capita (CAD, 2019)  
0                                  NaN  


In [ ]:
# drop rows with missing values
df = df_can_stats.dropna()
df.isnull().sum()

Province name                           0
Population (2021 Census)                0
Land area (km^2)                        0
Population per km^2                     0
GDP (million CAD, 2019)                 0
GDP per capita (CAD, 2019)              0
Market income per capita (CAD, 2019)    0
dtype: int64

In [ ]:
# check for duplicates
df_can_stats.duplicated().sum()

0

In [ ]:
# check distribution of numeric columns
df_can_stats.describe()

,Population per km^2
count,13.000000
mean,7.399083
std,8.405390
min,0.019629
25%,1.377950
50%,5.420989
75%,10.864588
max,27.142136


In [ ]:
# check distribution of categorical columns
df_can_stats.describe(include='O')

,Province name,Population (2021 Census),Land area (km^2),"GDP (million CAD, 2019)","GDP per capita (CAD, 2019)","Market income per capita (CAD, 2019)"
count,13,13,13,13,13,13
unique,13,13,13,13,12,13
top,Ontario,"14,223,942","908,699.33","891,811","54,149","36,298"
freq,1,1,1,1,2,1


In [ ]:
# check distribution of categorical columns using value_counts()
print(len(df_can_stats['Province name'].value_counts()))
# 13 matches the 10 Canadian provinces and 3 territories

13


# Examine Opioid and Stimulent Substance Harm Data